In [1]:
import pandas as pd
import ast
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

tqdm.pandas()

In [72]:
# 대용량 CSV 파일 경로
input_csv = r'C:\Users\bona_\infomax_crawler\labeled_edaily.csv'
train_csv = "train_data.csv"
test_csv = "test_data.csv"

# 청크 크기 설정
chunksize = 5000  # 조정 가능

def sentiment_label(x):
    return ast.literal_eval(x)

def make_model(nbc):
    if not nbc:
        return "", None
    
    tokens = []
    target = None

    for item in nbc:
        token, freq, label = item

        token_str = "_".join(token)
        if token_str not in tokens:
            tokens.append(token_str)

        if label == 1:
            target = "hawkish"
        elif label == -1:
            target = "dovish"

    document = " ".join(tokens) if tokens else ""
    return document, target

# 전체 데이터를 저장할 리스트 생성
document_list = []
target_list = []

# CSV 파일을 청크 단위로 읽기
for chunk in tqdm(pd.read_csv(input_csv, chunksize=chunksize), desc="Reading CSV"):
    # ngram_label 컬럼을 파싱하여 nbc 컬럼 생성
    chunk["nbc"] = chunk["ngram_label"].apply(sentiment_label)

    # make_model 적용 후 document, target 컬럼 생성
    chunk[["document", "target"]] = chunk["nbc"].apply(lambda x: pd.Series(make_model(x)))

    # 빈 문자열 & 중립("neutral") 값 제거
    chunk = chunk[(chunk["document"] != "") & (chunk["target"].notna())]

    # 리스트에 추가
    document_list.extend(chunk["document"].tolist())
    target_list.extend(chunk["target"].tolist())

# 전체 데이터 프레임 생성
full_data = pd.DataFrame({"document": document_list, "target": target_list})

# 데이터가 충분한지 확인
if len(full_data) < 2:
    print("데이터가 너무 적어 train_test_split을 실행할 수 없습니다.")
else:
    # 클래스가 하나뿐이면 stratify 사용 X
    class_counts = full_data["target"].value_counts()
    stratify_param = full_data["target"] if len(class_counts) > 1 else None

    # train-test split 실행
    X_train, X_test, y_train, y_test = train_test_split(
        full_data["document"], full_data["target"], test_size=0.1, random_state=42, stratify=stratify_param
    )

    # DataFrame 변환
    train_data = pd.DataFrame({"document": X_train, "target": y_train})
    test_data = pd.DataFrame({"document": X_test, "target": y_test})

    # CSV 파일 저장
    train_data.to_csv(train_csv, index=False)
    test_data.to_csv(test_csv, index=False)

    print("데이터 분리 완료! train_data.csv & test_data.csv 생성됨")


Reading CSV: 1it [03:24, 204.15s/it]


데이터 분리 완료! train_data.csv & test_data.csv 생성됨


In [73]:
# 저장된 train/test 데이터 불러오기
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")

X_train = train_data["document"].fillna("")  # 결측값 방지
y_train = train_data["target"].fillna("neutral")  # 결측값 방지
X_test = test_data["document"].fillna("")
y_test = test_data["target"].fillna("neutral")

In [74]:
# TfidfVectorizer로 문서 벡터화
vectorizer = TfidfVectorizer(ngram_range=(1,5), max_features=5000)
X_train_vec = vectorizer.fit_transform(tqdm(X_train, desc="Train_set TF-IDF 벡터화"))
X_test_vec = vectorizer.transform(tqdm(X_test, desc="Test_set TF-IDF 벡터화 진행"))

Test_set TF-IDF 벡터화 진행: 100%|██████████| 57/57 [00:00<00:00, 154.47it/s]


In [75]:
# 나이브 베이즈 학습(최초)
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)

# 모델 저장(이후 결과 누적 위해)
import joblib
joblib.dump(nb, "naive_bayes_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']

In [ ]:
###누적학습

In [77]:
# 기존 모델 및 벡터라이저 로드
try:
    nb = joblib.load("naive_bayes_model.pkl")
    vectorizer = joblib.load("vectorizer.pkl")
    print("기존 모델 및 벡터라이저 로드 완료")
except FileNotFoundError:
    print("기존 모델이 없음")
    exit()

# 새로운 CSV 파일
new_news_csv = r'C:\Users\bona_\infomax_crawler\labeled_economy.csv'
new_df = pd.read_csv(new_news_csv)

# `make_model()`을 적용하여 'ngram' -> `document`, `target` 생성
if "document" not in new_df.columns:
    if "ngram_label" in new_df.columns:
        print("ngram_label => document, target으로 변환 중")
        new_df[["document", "target"]] = new_df["ngram_label"].apply(lambda x: pd.Series(make_model(ast.literal_eval(x))))
    else:
        print("'document', 'ngram_label' 없음")
        exit()

# 데이터 확인 (중복 데이터 확인)
new_df = new_df[(new_df["document"] != "") & (new_df["target"].notna())]
print("새로운 데이터 개수:", len(new_df))

if len(new_df) == 0:
    print("새로운 데이터 존재X, 학습 종료료")
    exit()

# TF-IDF 변환 (기존 벡터라이저 사용)
X_new = vectorizer.transform(new_df["document"].fillna(""))
y_new = new_df["target"].astype(str)

# partial_fit 지원여부 확인인
if hasattr(nb, "partial_fit"):
    print("partial_fit 지원함. 추가 학습 진행")
    nb.partial_fit(X_new, y_new, classes=["hawkish", "dovish"])
else:
    print("partial_fit 지원 x. 학습 중단.")
    exit()

# 업데이트된 모델 저장
joblib.dump(nb, "naive_bayes_model.pkl")

# 모델 파일 크기 확인
import os
print("모델 파일 크기:", os.path.getsize("naive_bayes_model.pkl"))

print("모델 업데이트!")


기존 모델 및 벡터라이저 로드 완료
ngram_label => document, target으로 변환 중
새로운 데이터 개수: 410
partial_fit 지원함. 추가 학습 진행
모델 파일 크기: 160839
모델 업데이트!


In [67]:
# 최종
# 테스트 데이터 예측 및 평가
y_pred = nb.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      dovish       0.95      0.63      0.76       665
     hawkish       0.77      0.97      0.86       829

    accuracy                           0.82      1494
   macro avg       0.86      0.80      0.81      1494
weighted avg       0.85      0.82      0.81      1494

